In [1]:
# Libraries
import pandas as pd
import pycountry_convert as pc
import numpy as np
import geopy.distance

In [2]:
def get_airports_df():
    airports_df = pd.read_csv("../data/airports.csv", header=None)
    airports_df.columns = ["airport_id", "name", "city", "country", "IATA", "ICAO", "latitude", "longitude", 
                        "altitude", "time_zone", "dst", "tz_database_time_zone", "type", "source"]
    airports_df.drop(["airport_id", "ICAO", "altitude", "time_zone", "dst", "tz_database_time_zone", "type", "source"], axis = 1, inplace = True)
    return airports_df 

airports_df = get_airports_df()
airports_df

,name,city,country,IATA,latitude,longitude
0,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998
1,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005
3,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001
...,...,...,...,...,...,...
7693,Rogachyovo Air Base,Belaya,Russia,\N,71.616699,52.478298
7694,Ulan-Ude East Airport,Ulan Ude,Russia,\N,51.849998,107.737999
7695,Krechevitsy Air Base,Novgorod,Russia,\N,58.625000,31.385000
7696,Desierto de Atacama Airport,Copiapo,Chile,CPO,-27.261200,-70.779198


In [3]:
def get_continent_code():
    country_codes = []
    for _, row in airports_df.iterrows():
        try:
            country_code = pc.country_name_to_country_alpha2(row.country, cn_name_format="default")
            continent_code = pc.country_alpha2_to_continent_code(country_code)
            country_codes.append(continent_code)
        except:
            country_codes.append(np.nan)
    return country_codes

airports_df["continent_code"] = get_continent_code()

In [4]:
def drop_null_values():
    for column in airports_df:
        airports_df[column] = airports_df[column].replace([r"\N"],np.nan)
    airports_df.dropna(how="any", inplace=True)

drop_null_values()

In [5]:
airports_df

,name,city,country,IATA,latitude,longitude,continent_code
0,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998,OC
1,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001,OC
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005,OC
3,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977,OC
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001,OC
...,...,...,...,...,...,...,...
7684,Bilogai-Sugapa Airport,Sugapa-Papua Island,Indonesia,UGU,-3.739560,137.031998,AS
7687,Ramon Airport,Eilat,Israel,ETM,29.723694,35.011416,AS
7688,Rustaq Airport,Al Masna'ah,Oman,MNH,23.640556,57.487500,AS
7689,Laguindingan Airport,Cagayan de Oro City,Philippines,CGY,8.612203,124.456496,AS


In [6]:
airports_df.continent_code.value_counts()

NA    1941
AS    1263
EU    1061
SA     572
AF     567
OC     520
Name: continent_code, dtype: int64

In [7]:
def get_routes_df():
    routes = pd.read_csv("../data/routes.csv", header=None)
    routes.columns = ["airline", "airline_id", "source", "source_airport_id", "destination", 
                        "destination_airport_id", "codeshare", "stops", "equipment"]
    routes.drop(["airline", "airline_id", "source_airport_id", "destination_airport_id", "codeshare", "stops", "equipment"], axis = 1, inplace = True)
    routes.drop_duplicates(subset=["source", "destination"], inplace=True)
    return routes

routes = get_routes_df()
routes

,source,destination
0,AER,KZN
1,ASF,KZN
2,ASF,MRV
3,CEK,KZN
4,CEK,OVB
...,...,...
67653,TSV,WIN
67654,WGA,MEL
67656,WIN,LRE
67657,WIN,TSV


In [8]:
def merge_dataframes():
    routes_source = pd.merge(left=routes, right=airports_df[["IATA","latitude","longitude","continent_code"]], left_on="source", right_on="IATA")
    routes_source.drop(["IATA"], axis = 1, inplace = True)
    routes_source.rename(columns={"latitude":"source_lat", "longitude":"source_lon", "continent_code":"source_code"}, inplace=True)

    routes_df = pd.merge(left=routes_source, right=airports_df[["IATA","latitude","longitude","continent_code"]], left_on="destination", right_on="IATA")
    routes_df.drop(["IATA"], axis = 1, inplace = True)
    routes_df.rename(columns={"latitude":"dest_lat", "longitude":"dest_lon", "continent_code":"dest_code"}, inplace=True)
    
    return routes_df

routes_df = merge_dataframes()
routes_df

,source,destination,source_lat,source_lon,source_code,dest_lat,dest_lon,dest_code
0,AER,KZN,43.449902,39.956600,EU,55.606201,49.278702,EU
1,ASF,KZN,46.283298,48.006302,EU,55.606201,49.278702,EU
2,CEK,KZN,55.305801,61.503300,EU,55.606201,49.278702,EU
3,DME,KZN,55.408798,37.906300,EU,55.606201,49.278702,EU
4,EGO,KZN,50.643799,36.590099,EU,55.606201,49.278702,EU
...,...,...,...,...,...,...,...,...
36601,ZKE,YFA,52.282501,-81.677803,NA,52.201401,-81.696899,NA
36602,YPO,YAT,54.988098,-85.443298,NA,52.927502,-82.431900,NA
36603,ZKE,YAT,52.282501,-81.677803,NA,52.927502,-82.431900,NA
36604,JCK,RCM,-20.668301,141.723007,OC,-20.701900,143.115005,OC


In [9]:
def get_distance():  
    distance = []
    for i in range(len(routes_df)):
        source = (routes_df.source_lat[i], routes_df.source_lon[i])
        dest = (routes_df.dest_lat[i], routes_df.dest_lon[i])
        distance.append(geopy.distance.distance(source, dest).km)
    return np.round(distance, 2)

routes_df["distance"] = get_distance()
routes_df

,source,destination,source_lat,source_lon,source_code,dest_lat,dest_lon,dest_code,distance
0,AER,KZN,43.449902,39.956600,EU,55.606201,49.278702,EU,1507.99
1,ASF,KZN,46.283298,48.006302,EU,55.606201,49.278702,EU,1040.94
2,CEK,KZN,55.305801,61.503300,EU,55.606201,49.278702,EU,773.13
3,DME,KZN,55.408798,37.906300,EU,55.606201,49.278702,EU,718.08
4,EGO,KZN,50.643799,36.590099,EU,55.606201,49.278702,EU,1010.82
...,...,...,...,...,...,...,...,...,...
36601,ZKE,YFA,52.282501,-81.677803,NA,52.201401,-81.696899,NA,9.12
36602,YPO,YAT,54.988098,-85.443298,NA,52.927502,-82.431900,NA,302.73
36603,ZKE,YAT,52.282501,-81.677803,NA,52.927502,-82.431900,NA,88.10
36604,JCK,RCM,-20.668301,141.723007,OC,-20.701900,143.115005,OC,145.08
